# Import and clean Prescription data

In [1]:
#import required packages
#%pip install sklearn ipywidgets numpy pandas matplotlib requests ipywidgets IPython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib notebook

In [2]:
#load version information
%load_ext version_information

In [3]:
#Version for Pandas and numpy
#Pandas (1.2.4), numpy (1.20.1), matplotlib (3.3.4), requests (2.25.1), json (2.0.9) were used for this assignment
%version_information pandas, numpy, matplotlib, requests, json, ipywidgets, IPython, sklearn, scipy

Software,Version
Python,3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython,7.22.0
OS,Windows 10 10.0.22000 SP0
pandas,1.2.4
numpy,1.20.1
matplotlib,3.3.4
requests,2.25.1
json,2.0.9
ipywidgets,7.6.3
IPython,7.22.0


In [4]:
#list of functions

#function to get API data
def API_ping (API):
    api_response = requests.get(API)
    if api_response:
        print('request successful')
    else:
        print('request not successful')
        print(api_response.status_code)

#get charcteristics of a dataframe
def df_char(df):
    print('Shape of dataframe:', df.shape)
    print('\nColumn names:',df.columns, '\n')
    print( df.info())

#clean column names
def clean_columns(df):
    df.columns = df.columns.str.strip(" ")
    df.columns = df.columns.str.strip("_")
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('-', '_')
    df.columns = df.columns.str.replace('113_cause_name', 'cause_name_113')
    df.columns = df.columns.str.replace('Age_adjusted_Rate', 'AADR')
    df.columns = df.columns.str.replace('Number_of_Deaths', 'Deaths')
    df.columns = df.columns.str.replace('Abbreviation', 'Ab')
    print(df.columns)


#graph two data sets
def graph_me(df1, df2, PTitle, Label1, Label2):
    fig1 = plt.figure(1)
    plt. clf()
    plt.plot(df1.year,df1.population, label= Label1)
    plt.plot(df2.year,df2.population,label= Label2)
    plt.title(PTitle)
    plt.ylabel(df1.columns[1])
    plt.xlabel(df1.columns[0])
    #plt.xticks(ticks=[2000,2005,2010,2015])
    plt.legend()
    plt.show()
    
#graph two data sets for figure 3
def graph_me2(df1, df2, PTitle, Label1, Label2):
    df1 = df1.diff()
    df2 = df2.diff()
    fig3 = plt.figure(3)
    plt. clf()
    plt.plot(df1.year,df1.population, label= Label1)
    plt.plot(df2.year,df2.population,label= Label2)
    plt.title(PTitle)
    plt.ylabel(df1.columns[1])
    plt.xlabel(df1.columns[0])
    #plt.xticks(ticks=[2000,2005,2010,2015])
    plt.legend()
    plt.show()

#predict population based on area and year
#Most of the below is adapted from 
#https://medium.com/codex/stock-predication-using-regression-algorithm-in-python-fb8b426453b9
def linear_regression(df):
    x = df[['area', 'year']]
    y = df['population']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.15 , shuffle=False,random_state = 0)
    regression = LinearRegression()
    regression.fit(train_x, train_y)
    print('\n', "regression coefficient",regression.coef_)
    print('\n', "regression intercept",regression.intercept_)
    regression_confidence = regression.score(test_x, test_y)
    print('\n', "linear regression confidence: ", regression_confidence)
    predicted = regression.predict(test_x)
    dflr=pd.DataFrame({'Actual_Population':test_y, 'Predicted_Population':predicted})
    print(dflr.head(10))
    print(dflr.tail(10))
    print('\n', 'Mean Absolute Error (MAE):', metrics.mean_absolute_error(test_y, predicted))
    print('Mean Squared Error (MSE) :', metrics.mean_squared_error(test_y, predicted))
    print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(test_y, predicted)))
    x2 = dflr.Actual_Population.mean()
    y2 = dflr.Predicted_Population.mean()
    Accuracy1 = x2/y2*100
    print('\n', "The accuracy of the model is " , Accuracy1)
    #fig3 = plt.figure(3)
    #plt.scatter(dflr.Actual_Population, dflr.Predicted_Population,  color='Darkblue')
    #plt.xlabel("Actual Population")
    #plt.ylabel("Predicted Population")
    #plt.show()
    slope, intercept, r_value, p_value, std_err = linregress(dflr)
    #fig2 = plt.figure(2)
    #plt.plot(dflr.Actual_Population, color='black',label='Acutal')
    #plt.plot(dflr.Predicted_Population, color='lightblue',label='Predicted')
    #plt.plot((intercept + slope*dflr.Predicted_Population), 'r', label='fitted line')
    #plt.title("Population predicted")
    #plt.legend();
    #plt.show()
    prediction_population = []
    prediction_year = []
    for i in range(50):
        #print(i+1990)
        temp_pop = regression.coef_[1]*(i+1990) + regression.intercept_
        prediction_population.append(temp_pop)
        prediction_year.append(i+1990)
        #print(prediction_population)
        #print(prediction_year)
    df_Prediction = pd.DataFrame(prediction_population,prediction_year)
    #print(df_Prediction)
    fig2 = plt.figure(2)
    plt.plot(df_Prediction, color = 'black',label='predicted')
    plt.plot(df.year,df.population, color = 'blue', label = 'actual')
    plt.title("Population predicted")
    plt.legend()
    plt.xlabel('year')
    plt.ylabel('population')
    plt.show()
    
    return dflr

In [6]:
#import data
Overdose_Deaths = 'https://github.com/boward10/Opioid_Overdose_Dashboard_Project/blob/main/County_Opioid_Rate_Maps.xlsx'
print(Overdose_Deaths)
xlsx = pd.ExcelFile('https://github.com/boward10/Opioid_Overdose_Dashboard_Project/blob/main/County_Opioid_Rate_Maps.xlsx')
print(xlsx)
#df1 = pd.read_excel(xls, 'Sheet1')
#df2 = pd.read_excel(xls, 'Sheet2')
#AADR = pd.read_csv(Overdose_Deaths)
#print(AADR.head())

https://github.com/boward10/Opioid_Overdose_Dashboard_Project/blob/main/County_Opioid_Rate_Maps.xlsx


ValueError: File is not a recognized excel file